In [6]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd


import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [7]:
import geopy
from geopy.geocoders import Nominatim
!conda install -c conda-forge geopandas --yes
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cfitsio-3.470              |       hb60a0a2_2         1.4 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    bzip2-1.0.8                |       h516909a_2         396 KB  conda-forge
    rtree-0.9.4                |   py36he053a7a_1          43 KB  conda-forge
    xerces-c-3.2.2             |    hea5cb30_1003         1.7 MB  conda-forge
    postgresql-11.3            |       hbeee2d4_0         4.8 MB  conda-forge
    pcre-8.44                  |       he1b5a44_0         261 KB  conda-forge
    boost-cpp-1.68.0           |    h11c811c_1000        20.5 MB  conda-forge
    geos-3.8.0                 |       he1b5a44_1         1.0 MB  conda-forge
    fiona-1.8

poppler-0.65.0       | 1.6 MB    | ##################################### | 100% 
ca-certificates-2020 | 145 KB    | ##################################### | 100% 
libpq-11.3           | 2.5 MB    | ##################################### | 100% 
hdf4-4.2.13          | 969 KB    | ##################################### | 100% 
python_abi-3.6       | 4 KB      | ##################################### | 100% 
kealib-1.4.7         | 175 KB    | ##################################### | 100% 
openssl-1.1.1g       | 2.1 MB    | ##################################### | 100% 
proj-6.2.1           | 10.2 MB   | ##################################### | 100% 
geotiff-1.5.1        | 286 KB    | ##################################### | 100% 
pyproj-2.4.2.post1   | 347 KB    | ##################################### | 100% 
curl-7.68.0          | 137 KB    | ##################################### | 100% 
tzcode-2020a         | 425 KB    | ##################################### | 100% 
gdal-3.0.2           | 1.3 M

<b> Scraping the Los Angeles county city data </b>

In [8]:

url = 'https://en.wikipedia.org/wiki/List_of_cities_in_Los_Angeles_County,_California'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
#places_list = html_soup.find_all('table', class_='wikitable sortable')
places_list = html_soup.find('table',{'class':'wikitable sortable'})

In [9]:
column_names = ['City','Date','Population']

la = pd.DataFrame(columns = column_names)

try:
#loop through table, grab each of the 4 columns shown (try one of the links yourself to see the layout)
    for row in places_list.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) == 3:
            la.append(( cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
except: pass  

source = requests.get(url).text
la_data = BeautifulSoup(source, 'html')

# creat a new Dataframe
column_names = ['City','Date','Population']
la = pd.DataFrame(columns = column_names)
content = la_data.find('div', class_='mw-parser-output')
table = content.table.tbody
City = 0
Date = 0
Population = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            City = td.text
            i = i + 1
        elif i == 1:
            Date = td.text
            i = i + 1
        elif i == 2: 
            Population = td.text.strip('\n').replace(']','')
    la = la.append({'City': City,'Date': Date,'Population': Population},ignore_index=True)
la = la.shift(periods=-1, freq=None, axis=0, fill_value=None)
la = la.drop(88)
la = la.drop(columns = 'Date')
la['City'] = la['City'].astype(str) + ', Los Angeles'

la

,City,Population
0,"Agoura Hills, Los Angeles","20,330"
1,"Alhambra, Los Angeles","83,653"
2,"Arcadia, Los Angeles","56,364"
3,"Artesia, Los Angeles","16,522"
4,"Avalon, Los Angeles","3,728"
5,"Azusa, Los Angeles","46,361"
6,"Baldwin Park, Los Angeles","75,390"
7,"Bell, Los Angeles","35,477"
8,"Bell Gardens, Los Angeles","42,072"
9,"Bellflower, Los Angeles","76,616"


<b> Calling API </b>

In [10]:
CLIENT_ID = '2QK31LAJXGY0GDUVIHZYD2HS52U0YKSPSJAB5TEJGFA3JFAW' # your Foursquare ID
CLIENT_SECRET = 'KGJGADABBIUPC3AUOON3SOPELNTZWTHSCV4CMRTYDEFSTXRH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2QK31LAJXGY0GDUVIHZYD2HS52U0YKSPSJAB5TEJGFA3JFAW
CLIENT_SECRET:KGJGADABBIUPC3AUOON3SOPELNTZWTHSCV4CMRTYDEFSTXRH


In [11]:
address = 'Los Angeles, LA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# define a function to get coordinates
def get_latlng(City):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        address = City

        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print(latitude, longitude)
        lat_lng_coords = (latitude, longitude)
    return lat_lng_coords
coords = [ get_latlng(City) for City in la["City"].tolist() ]


34.14791 -118.7657042
34.093042 -118.12706
34.1362075 -118.0401497
33.8690197 -118.0796195
33.34221 -118.3272612
34.1338751 -117.9056046
34.0854739 -117.9611761
33.9747806 -118.1866361
33.9694561 -118.1503953
33.8825705 -118.1167679
34.0696501 -118.3963062
34.1469511 -117.9708982
34.1816482 -118.3258554
34.1446643 -118.6440973
33.8322043 -118.2517547
33.8644291 -118.0539323
34.0966764 -117.7197785
34.002581 -118.156586
33.894927 -118.226624
34.1024 -117.85486499999999
33.9620584 -118.1835395
34.0211224 -118.3964665
34.0286226 -117.8103367
33.942215 -118.1235646
34.1394513 -117.9772873
34.0536909 -118.2427666
33.917028 -118.4156337
33.8963593 -118.3053037
34.1469416 -118.2478471
34.1361187 -117.865339
33.8284787 -118.0743137
33.9188589 -118.3483256
33.86428 -118.39591
34.164091 -118.657837
33.9827043 -118.2120343
34.0197335 -117.9586754
33.9562003 -118.353132
34.1069927 -117.9342148
34.1998302 -118.2005236
33.9604546 -117.9504255
33.9060971 -118.0107092
34.01979 -117.9503677
34.1008426 

In [130]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
la['Latitude'] = df_coords['Latitude']
la['Longitude'] = df_coords['Longitude']
# save the DataFrame as CSV file
la = la.drop(columns="Population")
la

,City,Latitude,Longitude
0,"Agoura Hills, Los Angeles",34.147910,-118.765704
1,"Alhambra, Los Angeles",34.093042,-118.127060
2,"Arcadia, Los Angeles",34.136207,-118.040150
3,"Artesia, Los Angeles",33.869020,-118.079620
4,"Avalon, Los Angeles",33.342210,-118.327261
5,"Azusa, Los Angeles",34.133875,-117.905605
6,"Baldwin Park, Los Angeles",34.085474,-117.961176
7,"Bell, Los Angeles",33.974781,-118.186636
8,"Bell Gardens, Los Angeles",33.969456,-118.150395
9,"Bellflower, Los Angeles",33.882570,-118.116768


In [13]:
# get the coordinates of Los Angeles
address = 'Los Angeles, Los Angeles county, USA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of LA City, Los Angeles, USA {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of LA City, Los Angeles, USA 34.0536909, -118.2427666.


<b> Map of Los Angeles with the cities labeled </b>

In [15]:
# create map of Ls Angeles using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, City in zip(la['Latitude'], la['Longitude'], la['City']):
    label = '{}'.format(City)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_la)  
map_la

In [16]:
# save the map as HTML file
map_la.save('map_la.html')

In [162]:
from folium.plugins import HeatMap
from folium import plugins
import seaborn as sns


# Ensure you're handing it floats
la['Latitude'] = la['Latitude'].astype(float)
la['Longitude'] = la['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs

la_df = la[['Latitude', 'Longitude']]

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in la_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_la)
map_la


In [160]:
map_la.save('map_la.html')

<b> Getting venues near Los Angeles county </b>

In [20]:
radius = 1000

venues = []

for lat, lng, City in zip(la['Latitude'], la['Longitude'], la['City']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            City,
            
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

  


In [105]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['City','Name', 'Latitude', 'Longitude', 'Category']

print(venues_df.shape)
venues_df.head()

(2337, 5)


,City,Name,Latitude,Longitude,Category
0,"Agoura Hills, Los Angeles",El Pollo Loco,34.144732,-118.761088,Fast Food Restaurant
1,"Agoura Hills, Los Angeles",Ladyface Alehouse & Brasserie,34.143834,-118.762823,Brewery
2,"Agoura Hills, Los Angeles",Lal Mirch,34.147822,-118.760536,Indian Restaurant
3,"Agoura Hills, Los Angeles",Islands Restaurant,34.143774,-118.762049,American Restaurant
4,"Agoura Hills, Los Angeles",LA Fitness,34.146057,-118.766854,Gym


In [107]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add the venues as blue circle markers
for lat, lng, category in zip(venues_df.Latitude, venues_df.Longitude, venues_df.Category):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map


<b> Number of venues in each city of Los Angeles county </b>

In [108]:
venues_df.groupby('City').count()

,Name,Latitude,Longitude,Category
City,,,,
"Agoura Hills, Los Angeles",30,30,30,30
"Alhambra, Los Angeles",30,30,30,30
"Arcadia, Los Angeles",30,30,30,30
"Artesia, Los Angeles",30,30,30,30
"Avalon, Los Angeles",30,30,30,30
"Azusa, Los Angeles",30,30,30,30
"Baldwin Park, Los Angeles",30,30,30,30
"Bell Gardens, Los Angeles",23,23,23,23
"Bell, Los Angeles",30,30,30,30


In [110]:
# one hot encoding
la_onehot = pd.get_dummies(venues_df[['Category']], prefix="", prefix_sep="")

# add city column back to dataframe
la_onehot['City'] = venues_df['City'] 

# move city column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,City,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Spot,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Night Market,Nightclub,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stationery Store,Steakhouse,Storage Facility,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Agoura Hills, Los Angeles",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [111]:
la_onehot.shape

(2337, 266)

In [66]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 265 uniques categories.


In [112]:
la_grouped = la_onehot.groupby('City').mean().reset_index()
la_grouped

,City,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Spot,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Night Market,Nightclub,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stationery Store,Steakhouse,Storage Facility,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Agoura Hills, Los Angeles",0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,

In [113]:
num_top_venues = 10

for city in la_grouped['City']:
    print("----"+city+"----")
    temp = la_grouped[la_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agoura Hills, Los Angeles----
                  venue  freq
0  Fast Food Restaurant  0.10
1         Deli / Bodega  0.07
2        Breakfast Spot  0.07
3                   ATM  0.03
4            Sports Bar  0.03
5             Multiplex  0.03
6      Sushi Restaurant  0.03
7                   Gym  0.03
8         Grocery Store  0.03
9               Brewery  0.03


----Alhambra, Los Angeles----
                venue  freq
0              Bakery  0.10
1                Café  0.07
2        Burger Joint  0.07
3    Stationery Store  0.03
4  Burmese Restaurant  0.03
5       Grocery Store  0.03
6         Coffee Shop  0.03
7           Bookstore  0.03
8       Movie Theater  0.03
9  Seafood Restaurant  0.03


----Arcadia, Los Angeles----
                 venue  freq
0       Sandwich Place  0.10
1  American Restaurant  0.07
2       Clothing Store  0.07
3            Racetrack  0.07
4   Mexican Restaurant  0.07
5         Night Market  0.03
6        Movie Theater  0.03
7             Tea Room  0.03
8   

                       venue  freq
0       Fast Food Restaurant  0.13
1                   Pharmacy  0.07
2          Convenience Store  0.07
3  South American Restaurant  0.03
4               Burger Joint  0.03
5         Chinese Restaurant  0.03
6              Grocery Store  0.03
7                    Brewery  0.03
8              Bowling Alley  0.03
9         Mexican Restaurant  0.03


----Downey, Los Angeles----
                  venue  freq
0           Coffee Shop  0.10
1   American Restaurant  0.07
2      Sushi Restaurant  0.07
3  Fast Food Restaurant  0.07
4        Cosmetics Shop  0.07
5          Burger Joint  0.07
6      Department Store  0.03
7    Chinese Restaurant  0.03
8           High School  0.03
9           Supermarket  0.03


----Duarte, Los Angeles----
                  venue  freq
0  Fast Food Restaurant  0.17
1         Grocery Store  0.10
2           Pizza Place  0.10
3             Juice Bar  0.03
4                  Bank  0.03
5          Burger Joint  0.03
6    Chinese Re

                  venue  freq
0  Fast Food Restaurant  0.17
1        Discount Store  0.10
2           Coffee Shop  0.07
3   Japanese Restaurant  0.07
4     Convenience Store  0.07
5                  Bank  0.07
6        Sandwich Place  0.03
7    Mexican Restaurant  0.03
8       Thai Restaurant  0.03
9           Pizza Place  0.03


----Lancaster, Los Angeles----
                    venue  freq
0             Coffee Shop  0.09
1                   Plaza  0.09
2            Dessert Shop  0.04
3     Indie Movie Theater  0.04
4  Furniture / Home Store  0.04
5   Performing Arts Venue  0.04
6         Thai Restaurant  0.04
7      Mexican Restaurant  0.04
8           Hot Dog Joint  0.04
9                    Café  0.04


----Lawndale, Los Angeles----
                      venue  freq
0            Sandwich Place  0.10
1       Peruvian Restaurant  0.07
2               Coffee Shop  0.07
3  Mediterranean Restaurant  0.07
4        Mexican Restaurant  0.07
5              Noodle House  0.07
6      Fast Foo

9  Nature Preserve  0.00


----Rolling Hills, Los Angeles----
                           venue  freq
0                          Trail  0.33
1            Indie Movie Theater  0.33
2               Business Service  0.33
3                            ATM  0.00
4                Organic Grocery  0.00
5                           Park  0.00
6  Paper / Office Supplies Store  0.00
7                    Outlet Mall  0.00
8              Other Repair Shop  0.00
9           Other Great Outdoors  0.00


----Rosemead, Los Angeles----
                    venue  freq
0      Chinese Restaurant  0.13
1   Vietnamese Restaurant  0.10
2                Pharmacy  0.07
3      Mexican Restaurant  0.07
4    Fast Food Restaurant  0.07
5       Convenience Store  0.07
6             Pizza Place  0.03
7            Burger Joint  0.03
8                 Butcher  0.03
9  Furniture / Home Store  0.03


----San Dimas, Los Angeles----
                venue  freq
0  Italian Restaurant  0.07
1  Mexican Restaurant  0.07
2      S

In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = la_grouped['City']

for ind in np.arange(la_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agoura Hills, Los Angeles",Fast Food Restaurant,Deli / Bodega,Breakfast Spot,ATM,Sports Bar,Brewery,Burger Joint,Café,Shipping Store,Chinese Restaurant
1,"Alhambra, Los Angeles",Bakery,Burger Joint,Café,Grocery Store,Lounge,Market,Gastropub,Stationery Store,Bookstore,Movie Theater
2,"Arcadia, Los Angeles",Sandwich Place,American Restaurant,Clothing Store,Mexican Restaurant,Racetrack,Middle Eastern Restaurant,Music Venue,Fried Chicken Joint,Movie Theater,Stables
3,"Artesia, Los Angeles",Indian Restaurant,Sandwich Place,Ice Cream Shop,Skating Rink,Taiwanese Restaurant,Gift Shop,Gastropub,Bubble Tea Shop,Frozen Yogurt Shop,Shopping Mall
4,"Avalon, Los Angeles",Hotel,Seafood Restaurant,Bar,Mexican Restaurant,ATM,Cocktail Bar,Steakhouse,Brewery,Spa,Burger Joint


<b> Frequency of grocery stores in LA cities </b>

In [116]:
la_stops = la_grouped[["City","Grocery Store"]]
la_stops

,City,Grocery Store
0,"Agoura Hills, Los Angeles",0.033333
1,"Alhambra, Los Angeles",0.033333
2,"Arcadia, Los Angeles",0.000000
3,"Artesia, Los Angeles",0.033333
4,"Avalon, Los Angeles",0.000000
5,"Azusa, Los Angeles",0.033333
6,"Baldwin Park, Los Angeles",0.033333
7,"Bell Gardens, Los Angeles",0.043478
8,"Bell, Los Angeles",0.033333
9,"Bellflower, Los Angeles",0.033333


In [148]:
# set number of clusters
kclusters = 5

la_clustering = la_stops.drop(["City"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 0, 4, 0, 4, 4, 1, 4, 4], dtype=int32)

In [149]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
la_merged = la_stops.copy()

# add clustering labels
la_merged["Cluster Labels"] = kmeans.labels_

In [150]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(la.set_index("City"), on="City")
print(la_merged.shape)
la_merged.head() # check the last columns!

(88, 5)


,City,Grocery Store,Cluster Labels,Latitude,Longitude
0,"Agoura Hills, Los Angeles",0.033333,4,34.147910,-118.765704
1,"Alhambra, Los Angeles",0.033333,4,34.093042,-118.127060
2,"Arcadia, Los Angeles",0.000000,0,34.136207,-118.040150
3,"Artesia, Los Angeles",0.033333,4,33.869020,-118.079620
4,"Avalon, Los Angeles",0.000000,0,33.342210,-118.327261


In [151]:
# sort the results by Cluster Labels
print(la_merged.shape)
la_merged.sort_values(["Cluster Labels"], inplace=True)
la_merged

(88, 5)


,City,Grocery Store,Cluster Labels,Latitude,Longitude
43,"Lakewood, Los Angeles",0.000000,0,33.850346,-118.117191
59,"Paramount, Los Angeles",0.000000,0,33.898917,-118.171005
58,"Palos Verdes Estates, Los Angeles",0.000000,0,33.787239,-118.401813
57,"Palmdale, Los Angeles",0.000000,0,34.579313,-118.117111
56,"Norwalk, Los Angeles",0.000000,0,33.909280,-118.084917
55,"Monterey Park, Los Angeles",0.000000,0,34.051522,-118.129807
54,"Montebello, Los Angeles",0.000000,0,34.015940,-118.111975
51,"Manhattan Beach, Los Angeles",0.000000,0,33.891599,-118.395124
49,"Lynwood, Los Angeles",0.000000,0,33.924831,-118.202415
48,"Los Angeles, Los Angeles",0.000000,0,34.053691,-118.242767


<b> Cluster of grocery stores in LA cities</b>

In [152]:
# create map

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['City'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [153]:
la_merged.loc[la_merged['Cluster Labels'] == 0]


,City,Grocery Store,Cluster Labels,Latitude,Longitude
43,"Lakewood, Los Angeles",0.0,0,33.850346,-118.117191
59,"Paramount, Los Angeles",0.0,0,33.898917,-118.171005
58,"Palos Verdes Estates, Los Angeles",0.0,0,33.787239,-118.401813
57,"Palmdale, Los Angeles",0.0,0,34.579313,-118.117111
56,"Norwalk, Los Angeles",0.0,0,33.909280,-118.084917
55,"Monterey Park, Los Angeles",0.0,0,34.051522,-118.129807
54,"Montebello, Los Angeles",0.0,0,34.015940,-118.111975
51,"Manhattan Beach, Los Angeles",0.0,0,33.891599,-118.395124
49,"Lynwood, Los Angeles",0.0,0,33.924831,-118.202415
48,"Los Angeles, Los Angeles",0.0,0,34.053691,-118.242767


In [154]:
la_merged.loc[la_merged['Cluster Labels'] == 1]


,City,Grocery Store,Cluster Labels,Latitude,Longitude
7,"Bell Gardens, Los Angeles",0.043478,1,33.969456,-118.150395
82,"Vernon, Los Angeles",0.040000,1,34.007135,-118.225250


In [155]:
la_merged.loc[la_merged['Cluster Labels'] == 2]


,City,Grocery Store,Cluster Labels,Latitude,Longitude
24,"Duarte, Los Angeles",0.1,2,34.139451,-117.977287


In [156]:
la_merged.loc[la_merged['Cluster Labels'] == 3]


,City,Grocery Store,Cluster Labels,Latitude,Longitude
18,"Compton, Los Angeles",0.076923,3,33.894927,-118.226624
83,"Walnut, Los Angeles",0.066667,3,34.020289,-117.865339
84,"West Covina, Los Angeles",0.066667,3,34.068621,-117.938953
30,"Hawaiian Gardens, Los Angeles",0.066667,3,33.828479,-118.074314
11,"Bradbury, Los Angeles",0.066667,3,34.146951,-117.970898
29,"Glendora, Los Angeles",0.066667,3,34.136119,-117.865339
21,"Culver City, Los Angeles",0.066667,3,34.021122,-118.396467


In [157]:
la_merged.loc[la_merged['Cluster Labels'] == 4]


,City,Grocery Store,Cluster Labels,Latitude,Longitude
79,"South Pasadena, Los Angeles",0.033333,4,34.113306,-118.147829
5,"Azusa, Los Angeles",0.033333,4,34.133875,-117.905605
76,"Signal Hill, Los Angeles",0.033333,4,33.804826,-118.168259
3,"Artesia, Los Angeles",0.033333,4,33.869020,-118.079620
1,"Alhambra, Los Angeles",0.033333,4,34.093042,-118.127060
6,"Baldwin Park, Los Angeles",0.033333,4,34.085474,-117.961176
8,"Bell, Los Angeles",0.033333,4,33.974781,-118.186636
38,"La Cañada Flintridge, Los Angeles",0.033333,4,34.199830,-118.200524
10,"Beverly Hills, Los Angeles",0.033333,4,34.069650,-118.396306
12,"Burbank, Los Angeles",0.033333,4,34.181648,-118.325855


<b> Most of the grocery stores are clustered in Cluster 3 which is the Compton, Walnut, Culver City region and the next highest cluster is in Cluster 1, which is the Vernon region.</b>

<b> To avoid competition and ensure a good customer base, it is best if the grocery store is set up in cluster 0 which is mainly located on the coastal side of LA. </b>

<b> These places include Lakewood, Paramont, Santa Monics etc. Since these places are also high in population and tourist attractions, a grocery store in these regions would attract many people </b>